In [25]:
import os
import pickle as pkl
import dill
import numpy as np
import torch as th
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
for i in [0,1,2,3,4,5,6,7,9,10]:
    user = f'user_{i}'
    data_path = os.path.join(os.path.abspath(''), 'logs', user)
    zeros = []
    for exp in list(os.listdir(data_path)):
        try:
            data_file = os.path.join(data_path,exp,'data.pkl')
            with open(data_file, 'rb') as f:
                data = pkl.load(f)
        except:
            continue
        data = sum(data,[])

        for datum in data:
            gaze = [step['gaze_features'] for step in datum['observations']]
            zeros += [np.count_nonzero(feat) == 0 for feat in gaze]
    print(user,np.mean(zeros))

user_0 0.004202757418891907
user_1 0.0
user_2 0.04134705332086062
user_3 0.00029157352512391874
user_4 0.005872651944813161
user_5 0.03495544893762851
user_6 0.6076576353459805
user_7 0.8164115923152068
user_9 0.00014778325123152708
user_10 1.0


In [7]:
user = f'user_1'
data_path = os.path.join(os.path.abspath(''), 'logs', user)
zeros = []
exp = list(os.listdir(data_path))[2]
data_file = os.path.join(data_path,exp,'data.pkl')
with open(data_file, 'rb') as f:
    data = pkl.load(f)
data = sum(data,[])
params_file = os.path.join(data_path,exp,'params.pkl')
loaded = th.load(params_file, map_location='cpu')
encoder = loaded['trainer/vae'].encoder

In [8]:
encoder

Mlp(
  (fc0): Linear(in_features=162, out_features=64, bias=True)
  (last_fc): Linear(in_features=64, out_features=6, bias=True)
)

In [22]:
episodes = []
for episode in data:
    target = episode['env_infos'][0]['target_index']
    encodings = []
    for state in episode['observations']:
        obs = np.concatenate((state['base_obs'],*state['goal_set'],state['gaze_features']))
        obs = th.tensor(obs).float()
        encoding = encoder(obs)[:3].detach().numpy()
        encodings.append(encoding)
    encodings = np.array(encodings)
    episodes.append((target,encodings))

In [ ]:
latents = episodes[0][1]
fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111,projection='3d')

xs = latents[:, 0]
ys = latents[:, 1]
zs = latents[:, 2]
time = np.array(len(latents))[None]

colmap = cm.ScalarMappable(cmap=cm.plasma)
colmap.set_array(time)

yg = ax.scatter(xs, ys, zs, c=cm.plasma(time/max(time)), marker='o', s=50)
cb = fig.colorbar(colmap)

for i in range(len(xs) - 1):
    ax.plot([xs[i],xs[i+1]],[ys[i],ys[i+1]],[zs[i],zs[i+1]], color='k', alpha=0.1)

plt.show()

                    
plot_latents('switch', '3', 1, 3)
